In [115]:
import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


from face import FACE
from agnostic_face import FACE as AGNOSTIC_FACE


import pickle
import matplotlib.pyplot as plt

## Notes

Edge weight = feasbility + actionability

where,

feasbility: statistical - how easy is it to get from data point A to data point B. Modelled by some distance measure (and density estimation)

actionability: something subjective - how easy is it for individual A to become individual B
(modelled by user specified constraints)

distance measure: feasability, start with categorical variables and apply to continuous

### Counterfactual Generation: The Importance of Distance Measure

In its simplest form, a counterfactual generation algorithm receives as input: an instance to be explained, the set of n available training examples (instances) alongside their associated labels and the associated label of the instance to be explained. The counterfactual generation algorithm uses a distance function to determine how close input to be explained is to each training instance, and searches for the *optimal* instance with the opposite label.

In recent explainable AI literature, counterfactual generation algorithms have been expanded to explain model behaviour. In this case, the counterfactual generation algorithm is initialised as above with the additional input of the machine learning model trained on the training instances. As the counterfactuals are generated to explain the behaviour of the machine learing model the model predictions are used as instance labels instead of the ground truth labels. 

The importance of distance measure selection has been widely studied across many machine learning domains where it has been shown that the choice of measure drastically impacts the bias and generalisation capability of a machine learning algorithm.

TODO: WHY IS DISTANCE MEASURE IMPORTANT FOR COUNTERFACTUALS

### Distance Measures and Feature Types

The input instances are represented by a vector of features which can include:

Categorical features - describe variables that can take a fixed number of values. These features can be further subdivided into nominal features or ordinal features where ordinal features differ to nominal features by having a  natural ordering over the set of values 
Continuous features - describe variables that can have an infinite number of values whith a natural ordering.

Popular distance measures on Continuous Features:
<ul>
    <li> Euclidean distance (L2 norm):
        $ d(x,y) = \sqrt{(x-y)^2} $ </li>
    <li> Manhattan distance (L1 norm): 
        $ d(x,y) = |x-y| $ </li>
    Both can be generalised by the Minkowski distance:
    <li> Minkowski distance:
        $d(x,y) = (|x-y|^p)^\frac{1}{p} $ </li>
    which, in the limit as $p -> \inf$ gives the Chebyshev metric
    <li> Chebyshev distance:
        $d(x,y) = max(x_{i},y_{i}$ </li> </ul>

L norms each have desirable characteristics when applied as similarity measures on continuous features due to the metric space implicity defined by numerical features. When the objects are defined by a set of numerical attributes, there are natural definitions of distance based on geometric analogies. However, in the case of categorical data, there is a lack of metric space and there is no single ordering for the categorical values. For example consider how to define distance between different occupations. By defining a metric space over discrete features we implictly encode a specific set of assumptions. Distance measures applied to categorical features include: 

<ul>
    <li> One-hot encoding of categorical variables to treat as a binary (numerical) variable </li>
    <li> Value Difference Metric (uses probabilities over features) </li>
    
</ul>


The importance of distance measure selection has been widely studied across many machine learning domains where it has been shown that the choice of measure drastically impacts the bias and generalisation capability of a machine learning algorithm. There have been multiple distance measures proposed in the literature with varying properties with the most popoular measure being the Euclidean distance measure.  

The most popular distance measures are targeted towards continuous features. There have been considerably less distance measures proposed to handle categorical features and even less distance measures proposed to handle a mixture of both data types. Many real-world applications have both nominal and linear attributes, including, for
example, over half of the datasets in the UCI Machine Learning Database Repository. This motivates the need for distance measures that are capable of appropriately handling different kinds of features. 


TODO Counterfactual mixed features


### Approaches to Mixed Feature Distances

The most common approach to calculating distances between mixed feature types in counterfactual generation is by first transforming categorical features into binary features via one-hot-encoding such that they are able to be handled by conitnuous distance measures, principally the Euclidean distance.  

TODO what is one-hot encoding. 

### Vanilla Counetrfactual Generation: The Effect of One-Hot-Encoding and Euclidean Distance on FACE Counterfactuals

We run through an example of counterfactual generation for the adult dataset - widely used to evaluate counterfactuals. We use the FACE algorithm with the Euclidean distance measure. The adult dataset is preprocessed according to existing approaches to counterfactual generation. This included one-hot-encoding cateorical features

In [116]:
#### Import Data

In [117]:
data = pd.read_csv("adult.csv")
data = data[(data != '?').all(axis=1)].reset_index()
data

,index,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45217,48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
45218,48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
45219,48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
45220,48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


#### Handling different types of feature

How do we take into acount the dependency between features - is a woman who is 45 and pregnant more similar to a 25 year old who is also pregnant or to a 45 year old woman who is not pregnant? 


#### Select subset of 15 instances from dataset

In [118]:
subset_data = data.iloc[50:80]
subset_data

,index,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
50,56,63,Private,145985,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
51,57,34,Local-gov,382078,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,3103,0,50,United-States,>50K
52,58,42,Self-emp-inc,170721,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,50,United-States,>50K
53,59,33,Private,269705,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,40,United-States,<=50K
54,60,30,Private,101135,Bachelors,13,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,50,United-States,<=50K
55,61,39,Private,118429,Some-college,10,Divorced,Sales,Not-in-family,White,Male,0,0,40,United-States,<=50K
56,62,26,Private,31208,Masters,14,Never-married,Exec-managerial,Not-in-family,White,Female,0,0,40,United-States,<=50K
57,63,33,Private,281384,HS-grad,9,Never-married,Machine-op-inspct,Own-child,White,Female,0,0,40,United-States,<=50K
58,64,47,Local-gov,171807,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,<=50K
59,66,41,Self-emp-inc,445382,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,15024,0,60,United-States,>50K


### Feature Selection

To make our counterfactuals easy to understand include the following features: 

<ul>
    <li> age (Continuous) </li>
    <li> education (Ordinal) </li>
    <li> occupation (Nominal) </li>

</ul>


In [119]:
ordinal_indexes = [0,2]
nominal_indexes = [1]

In [120]:
data_selected = subset_data[['age','education','occupation','income']]
data_selected = data_selected.reset_index().drop("index",axis="columns")
data_selected

,age,education,occupation,income
0,63,HS-grad,Craft-repair,<=50K
1,34,Bachelors,Exec-managerial,>50K
2,42,HS-grad,Exec-managerial,>50K
3,33,HS-grad,Handlers-cleaners,<=50K
4,30,Bachelors,Exec-managerial,<=50K
5,39,Some-college,Sales,<=50K
6,26,Masters,Exec-managerial,<=50K
7,33,HS-grad,Machine-op-inspct,<=50K
8,47,HS-grad,Adm-clerical,<=50K
9,41,Assoc-acdm,Craft-repair,>50K


#### Preprocess data

Treat age and education as continuous and one-hot-encode occupation

In [121]:
data_encoded = pd.get_dummies(data_selected.occupation, prefix='occupation')
data_encoded["age"] = data_selected["age"]
data_encoded["education"] = data_selected["education"]
data_encoded["outcome"] = data_selected["income"]

education_dummies = {"education": {
    'Preschool' : 0,
    '1st-4th': 1,
    '5th-6th': 2,
    '7th-8th':3,
    '9th':4,
    '10th':5,
    '11th':6,
    '12th':7,
    'HS-grad':8,
    'Some-college':9,
    'Prof-school':10,
    'Assoc-acdm':11,
    'Assoc-voc':12,
    'Bachelors':13,
    'Masters':14,
    'Doctorate':15,
}} 
data_encoded = data_encoded.replace(education_dummies)
data_encoded["outcome"] = data_encoded["outcome"].astype('category')
data_encoded["outcome"] = data_encoded["outcome"].cat.codes
data_encoded

,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,age,education,outcome
0,0,0,1,0,0,0,0,0,0,0,63,8,0
1,0,0,0,1,0,0,0,0,0,0,34,13,1
2,0,0,0,1,0,0,0,0,0,0,42,8,1
3,0,0,0,0,1,0,0,0,0,0,33,8,0
4,0,0,0,1,0,0,0,0,0,0,30,13,0
5,0,0,0,0,0,0,0,1,0,0,39,9,0
6,0,0,0,1,0,0,0,0,0,0,26,14,0
7,0,0,0,0,0,1,0,0,0,0,33,8,0
8,1,0,0,0,0,0,0,0,0,0,47,8,0
9,0,0,1,0,0,0,0,0,0,0,41,11,1


In [122]:
data_encoded_Y = data_encoded['outcome']
data_encoded_X = data_encoded.drop(["outcome"], axis="columns")

#### Run the FACE Algorithm to generate the counterfactuals for the following instance

In [123]:
example = data_encoded_X.iloc[[2]]
example

,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,age,education
2,0,0,0,1,0,0,0,0,0,0,42,8


#### Generate counterfactual graph and print the path from example to nearest counterfactual

In [124]:
ce = AGNOSTIC_FACE(data_encoded_X, data_encoded_Y, ordinal_indexes, nominal_indexes, dist_metric="euclidean")
eg = example.values.reshape(1, -1)
path = ce.generate_counterfactual(eg,1)
path

euclidean
30 nodes have been added to graph.
157 edges have been added to graph.


,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,age,education
0,0,0,0,1,0,0,0,0,0,0,42,8
1,0,0,0,0,0,0,0,1,0,0,43,8


#### Evaluate Counterfactual

In [125]:
### alternative counterfactual instances

alternative_counterfactuals = data_encoded.loc[data_encoded.outcome == 0]
alternative_counterfactuals

,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,age,education,outcome
0,0,0,1,0,0,0,0,0,0,0,63,8,0
3,0,0,0,0,1,0,0,0,0,0,33,8,0
4,0,0,0,1,0,0,0,0,0,0,30,13,0
5,0,0,0,0,0,0,0,1,0,0,39,9,0
6,0,0,0,1,0,0,0,0,0,0,26,14,0
7,0,0,0,0,0,1,0,0,0,0,33,8,0
8,1,0,0,0,0,0,0,0,0,0,47,8,0
10,0,0,0,0,0,0,1,0,0,0,19,9,0
11,0,0,0,0,0,0,0,0,0,1,46,8,0
12,0,0,0,0,0,0,0,1,0,0,43,8,0


In [126]:
path

,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,age,education
0,0,0,0,1,0,0,0,0,0,0,42,8
1,0,0,0,0,0,0,0,1,0,0,43,8


The counterfactual generated by the Euclidean distance measure heavily weights the continuous attributes (age, education) over the binarised occupation attribute, giving a counterfactual instance where age and education level remain similar to the original instance yet occupation has been changed from "Exec managerial" to "Sales". The semantic significance of this change in occupation could render this counterfactual meaningless to an end user. Futhermore, if we examine the list of alternative counterfactual instances in the dataset we can find two instances where occupation is "Exec Managerial" with associated age and education levels of 30, 13 and 26, 14 and 23, 13 respectively. 

Using domain knowledge, we know that education level and occupuation are interconnected concepts and from the data we can see that the Exec managerial occupation is linked to higher education level. A more meaningful counterfcatual instance would reflect this relationship which is ignored by the Euclidean distance measure. 

## Our Proposed Counterfactual Distance Measure 

From Wilson et al. (https://arxiv.org/pdf/cs/9701101.pdf) who argue "that any value stored in a computer is discrete at some level. We adopt an alternative distance measure where we first transform continuous features into categorical features via binning. From this we obtain an input feature vector including nominal and ordinal variables.



It is a challenging task to reasonably define a distance of
mixed-categorical data because the relationship among categories of ordinal and nominal attributes exists in different
ways, which yields different types of intercategory distance

#### Transform orignal data into categorical features 

In [127]:
data_categorised = pd.DataFrame()
data_categorised['age'] = pd.qcut(data_selected[['age'][0]], 5, labels=False).values
data_categorised['education'] = data_encoded['education'].values
data_categorised["raw_occupation"] = data_selected["occupation"].astype('category').values
data_categorised["occupation"] = data_categorised["raw_occupation"].cat.codes
data_categorised["outcome"] = data_encoded["outcome"].values

data_categorised.drop(["raw_occupation"],axis="columns")


,age,education,occupation,outcome
0,4,8,2,0
1,2,13,3,1
2,2,8,3,1
3,1,8,4,0
4,1,13,3,0
5,2,9,7,0
6,1,14,3,0
7,1,8,5,0
8,4,8,0,0
9,2,11,2,1


### How to measure distance between categorical features

Our categorical distance measure is built on the intuition that two categories
containing more different information are usually more dissimilar to each other. For *feasible* counterfactual generation this assumption is useful as we take the representation of the data into account when determining similarity between instances such that instances with more features in common are closer than those without. We illustrate this intuition below. 


Lets examine a really small sample from the adult dataset transformed into its categorical form. Age and Education are ordinal features and education is a nominal feature. 



In [128]:
data_categorised.iloc[[0,2,4]]

,age,education,raw_occupation,occupation,outcome
0,4,8,Craft-repair,2,0
2,2,8,Exec-managerial,3,1
4,1,13,Exec-managerial,3,0




Intuitively, when determining the similarity between age categories, the dissimilarity of age category 2 and 1 is lower than that of 2 and 4, given occupation as these age categories have a common corresponding occupation value,32 where 2 and 4 do not.  
Also, the dissimilarity of age category 2 and 1 is higher than that of 2 and 4, given education as the corresponding education values of age categories 2 and 1 (8,13) are with larger order difference than that of age categories 2 and 4 (8,8). From this we can see that the ordinal education feature and the occupation feature indicate distance differently. 

By quantifying the information of categories indicated by different attributes using entropy, the distances between oridnal and nominal features are unified into homogeneous concepts and can thus be directly combined without causing information loss. We adopt the distance measure proposed by Zhang et al. 

The distance measure algorithm proposed by Zhang et al. is as follows: 

For a pair of instances X, Y each characterised by a d dimensional feature vector where d_o represent the ordinal feature indexes and d_n represent the nominal feature indexes:
    for i in d:
        for j in d: 
            1. Calculate the interdepdence between feature A_i and A_j 
            2. Calculate the entropy based distance between X_i and Y_i according to feature A_j
         3. Calculate the overall distance between X_i and Y_i
    4. Calculate the overall distance between X and Y 


1 - Incoporating a measure of the interdependence between two features into the distance measure is based on the intuition that if there is low interdepence between two features, A_i and A_j then the distribution over A_j should have limited influence on the distance between categories in A_i. The feature interdepedence manifests according to the feature type of A_i and A_j and is thus calculated in different ways:
    <ul>
    <li> Case 1: A_i and A_j are both nominal OR A_i and A_j are mixed: In this case, the interdependence is calculated by determining the proportion of instance pairs where X_i == Y_i and X_j == Y_j. </li>
        <li>Case 2: A_i and A_j are ordinal: In this case, the interdependence is calculated by determining the proportion of instance pairs where $(X_i > Y_i and X_j > Y_j)$ Or $(X_i < Y_i and X_i < Y_i)$ . </li>
    </ul>
2 - The entropy based distance measure is based on the intuition that larger amount
of different information contained by two different categories indicates that they are more dissimilar. The distance between two instance feature values X_i, Y_i given feature A_j is calculated by first taking the entropy over all joint probability distributions given by the following equation: 
    $E(X_i, Y_i, A_j) = - \Sigma^{d}_{u=1} p(XY_i, A_j) log_2(p(XY_i, A_j)) $
Then distance is determined based on feature types such that


if i in d_o: 
$d(X_i, Y_i, A_j)  = \frac{\Sigma_G = min(X_i,Y_i)^max(X_i,Y_i) E(X_i, G_i, A_s)}{S_A_j}$
if i in d_n:
$d(X_i, Y_i, A_j)  = \frac{E(X_i, Y_i, A_s)}{S_A_j}$

where S_A_j is the normalising Standard Information value. 

3 - To caluclate the overall distance between X_i and Y_i the interdepence measure is multplied by the entropic distance and summed over over each A_j

4 - To calculate the overall distance between X and Y each feature distance is squared and summed over all features and the overall distance is squarerooted. 




In [129]:
data_categorised_X = data_categorised[['age','occupation','education']]

data_categorised_Y = data_categorised['outcome']

In [130]:
### Generate Counterfactuals for example 

In [131]:
example = data_categorised_X.iloc[[2]]

In [132]:
from agnostic_face import FACE as AGNOSTIC_FACE
ce = AGNOSTIC_FACE(data_categorised_X, data_categorised_Y, ordinal_indexes, nominal_indexes, "categorical")

eg = example.values.reshape(1, -1)
path = ce.generate_counterfactual(eg, 1)

path

categorical
30 nodes have been added to graph.
435 edges have been added to graph.


,age,occupation,education
0,2,3,8
1,2,5,8


In [133]:
alternative_counterfactuals = data_categorised.loc[data_encoded.outcome == 0]
alternative_counterfactuals

,age,education,raw_occupation,occupation,outcome
0,4,8,Craft-repair,2,0
3,1,8,Handlers-cleaners,4,0
4,1,13,Exec-managerial,3,0
5,2,9,Sales,7,0
6,1,14,Exec-managerial,3,0
7,1,8,Machine-op-inspct,5,0
8,4,8,Adm-clerical,0,0
10,0,9,Other-service,6,0
11,3,8,Transport-moving,9,0
12,3,8,Sales,7,0


In [134]:
evaluate = data_selected.reset_index()
evaluate.iloc[[2,12,18]]

,index,age,education,occupation,income
2,2,42,HS-grad,Exec-managerial,>50K
12,12,43,HS-grad,Sales,<=50K
18,18,41,HS-grad,Machine-op-inspct,<=50K


### Evaluating Generated Counterfactuals

The above table shows the original instance, the generated counterfactual under the Euclidean distance measure and the generated counterfactual under UDM. We can see that both counterfactuals change the occupation of the original instance, maintain the education level and have equal changes to the age variable. If we explore the data further, the table below shows how the Machine Operator Inspector occupation have more instances which are similar across the age and education features to the original instance than the Sales occupation. This shows how the Unified Distance Measure generates counterfactuals which are more true to the distribution of the underlying data than the Euclidean distance measure.  

In [135]:
data_categorised

,age,education,raw_occupation,occupation,outcome
0,4,8,Craft-repair,2,0
1,2,13,Exec-managerial,3,1
2,2,8,Exec-managerial,3,1
3,1,8,Handlers-cleaners,4,0
4,1,13,Exec-managerial,3,0
5,2,9,Sales,7,0
6,1,14,Exec-managerial,3,0
7,1,8,Machine-op-inspct,5,0
8,4,8,Adm-clerical,0,0
9,2,11,Craft-repair,2,1


### Improving Distance Measure with Constraint Based Rules

#### Distance Metrics - The Case for breaking Metric Properties

A metric on a set $X$ is a function:
$$ d : X × X → R+ $$
For all $x, y, z$ in $X$, this function is required to satisfy the following conditions:
<ul>
    <li> 1. $ d(x, y) ≥ 0 $ </li>
    <li> 2. $d(x, y) = 0$  iff   $x = y$  </li>
    <li> 3. $ d(x, y) = d(y, x) $   </li>
    <li> 4. $ d(x, z) ≤ d(x, y) + d(y, z) $ </li>
</ul>
    
    
Why might these be violated when generating feasible counterfactual explanations?
<ul> 
    <li> 1. Non-negativity </li>
    <li> 2. identity of indiscernibles </li>
    <li> 3. Symmetry - Easy to increase a feature but impossible to decrease (i.e. age) </li>
    <li> 4. Triange - Easier to go "the long way round" rather than direct jump (i.e education level - impossible to go from kindergarten to PhD)</li>
    
Conjunction of 1 and 2 produce positive definiteness - important for convexity 

Extensions to Distance Metrics:
    <ul>
        <li> Pseudometrics satisfy all properties apart from 2 such that $d(x,x) = 0 $ and <it> possibly </it> $d(x,y)=0$. Similarly, metametrics satisfy all properties other than 2 such that $d(x,x)$ is not necessarily 0. </li> 
        <li> Quasimetrics obey by all properties other than 3. </li>
        <li> Semimetrics obey by all propoeties other than 4. </li>
        